In [1]:
import numpy as np
import string
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Администратор\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def tokenization(text):
    result = []
    r = re.compile("^[а-яА-ЯёЁ]+$") 

    for line in text:
        
        line = line.lower()
        line = re.sub(r'[^\w\s]','', line)
        line = [t for t in line.split() if t not in stopwords.words('russian')]
        temp = [word for word in filter(r.match, line)]
        result += temp

    return result

In [3]:
file_number = 1
index = {}
morph = pymorphy2.MorphAnalyzer(lang='ru')

while file_number<=165:
    with open('files/file_{0}.txt'.format(file_number), encoding='utf-8') as file:
        text = [row.strip() for row in file]
        tokens = tokenization(text)
        for item in tokens:
            lemma = morph.parse(item)[0].normal_form
            if lemma in index:
                if file_number not in index[lemma]:
                    index[lemma].append(file_number)
            else:
                index[lemma] = [file_number]
    file_number = file_number + 1

In [4]:
with open('files/inverted_index.txt','a') as file:
    for key,val in index.items():
        file.write('\n{} '.format(key))
        for item in val:
            file.write('{} '.format(item))

In [5]:
file = open("files/inverted_index.txt")
onstring = file.read().split("\n")[:-1]

inverted_index = dict()

for item in onstring:
    key = item.split(" ")[0]
    value = item.split(" ")[1:]
    inverted_index[key] = value

file.close()

In [6]:
query="мороженое OR молоко AND NOT холод"

result_set=set()
operation=None

for word in re.split(" +(AND|OR) +",query):

    inverted=False

    if word in ['AND','OR']:
        operation=word
        continue

    if word.find('NOT ') == 0:
        if operation == 'OR':
            continue

        inverted=True
        realword=word[4:]
        print(realword)
    else:
        realword=word
        print(realword)

    if operation is not None:
        current_set=set(inverted_index[realword])

        if operation == 'AND':
            if inverted == True:
                result_set -= current_set
            else:
                result_set &= current_set
        elif operation == 'OR':
            result_set |= current_set

    operation=None

print("result: ")
print(result_set)

мороженое
молоко
холод
result: 
{'72', '5', '15', '20', '95', '46', '76', '159', '126', '101', '79', '39', '138', '160', '84', '140', '35', '52', '111', '136', '105', '57', '63', '118', '60'}
